In [1]:
import pandas as pd

In [2]:
import numpy as np
import re
import pandas as pd
from tqdm import tqdm
from datasets import load_dataset
import umap
import altair as alt
from sklearn.metrics.pairwise import cosine_similarity
from annoy import AnnoyIndex
import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_colwidth', None)

In [4]:
#df = pd.read_csv('Insights.csv')
df = pd.read_csv('Insights_en.csv', sep=';')

df.columns = ['Insight', 'InsightID', 'InsightParentID', 'IsParent']
#df.head()

In [5]:
filter_no_parents = df['InsightParentID'] != df['InsightParentID']
fltered_df = df[filter_no_parents]
fltered_df['InsightParentID'] = fltered_df['InsightID']
df[filter_no_parents] = fltered_df


In [6]:
df["InsightParent"] = df["InsightParentID"].apply(lambda x: df.iloc[df['InsightID'].eq(x).idxmax()]["Insight"])

In [7]:
df.describe()

,Insight,InsightID,InsightParentID,IsParent,InsightParent
count,297,297,297,297,297
unique,297,297,70,2,70
top,"Temporary workers lack discipline in terms of punctuality, professionalism, and attention to detail, which results in costly mistakes and inappropriate behavior at work.",1702251558470x613868062724425100,1702251795709x628710337044019200,No,"Customers greatly appreciate the professionalism, the quality of listening and the excellence of the interactions with the Randstad advisor, reflecting highly satisfactory customer service."
freq,1,1,33,254,33


In [8]:
from sentence_transformers import SentenceTransformer
#model = SentenceTransformer('OrdalieTech/Solon-embeddings-large-0.1')
#model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-mpnet-base-v2')
#model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')
model = SentenceTransformer('intfloat/e5-mistral-7b-instruct', device='cpu')

No sentence-transformers model found with name /home/maitre/.cache/torch/sentence_transformers/intfloat_e5-mistral-7b-instruct. Creating a new one with MEAN pooling.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [9]:
sentences = df['Insight']
sentences[:3]

0     Temporary workers lack discipline in terms of punctuality, professionalism, and attention to detail, which results in costly mistakes and inappropriate behavior at work.
1                    Candidates report not being informed about the company, its business and job details before the interview, creating a lack of suitability and preparation.
2                                                              The client suggests setting up reference checks for new temporary workers who have never worked with the agency.
Name: Insight, dtype: object

In [11]:
sentences = df['Insight']
sentence_embeddings = model.encode(sentences)
sentence_embeddings.shape

(297, 4096)

In [12]:
sentence_embeddings

array([[-0.9971162 ,  1.9342052 , -1.3829827 , ...,  0.6241533 ,
        -3.119987  , -2.3335843 ],
       [ 1.4251552 ,  1.1378524 , -0.5419341 , ...,  2.6888282 ,
        -2.759878  , -1.1475271 ],
       [-1.3898997 ,  3.71497   , -0.30208707, ...,  2.298717  ,
         0.83176595,  0.25596514],
       ...,
       [-0.19075626,  2.4850957 ,  0.34755987, ...,  3.1810887 ,
         0.70644456,  0.5367545 ],
       [ 0.3489846 ,  4.093657  ,  2.1926184 , ...,  1.8694532 ,
        -2.848639  , -0.9706781 ],
       [ 1.820222  ,  3.8000915 ,  1.2423855 , ...,  1.5167245 ,
        -2.7281287 , -0.8079519 ]], dtype=float32)

In [13]:
#@Insight Plot the archive {display-mode: "form"}

# UMAP reduces the dimensions from 1024 to 2 dimensions that we can plot
reducer = umap.UMAP(n_neighbors=15)
umap_embeds = reducer.fit_transform(sentence_embeddings)
# Prepare the data to plot and interactive visualization
# using Altair
df_explore = pd.DataFrame(data={'text': df['Insight'], "InsightParent": df['InsightParent']})
df_explore['x'] = umap_embeds[:,0]
df_explore['y'] = umap_embeds[:,1]
df_explore


,text,InsightParent,x,y
0,"Temporary workers lack discipline in terms of punctuality, professionalism, and attention to detail, which results in costly mistakes and inappropriate behavior at work.","Temporary workers present deficiencies in terms of discipline at work, impacting the quality of their performance and their compliance with processes.",7.751192,0.942769
1,"Candidates report not being informed about the company, its business and job details before the interview, creating a lack of suitability and preparation.","Clients are dissatisfied with the selection of temporary workers, linked to candidates feeling poorly informed about the positions before the interview, leading to a profile/position mismatch.",7.273573,2.097942
2,The client suggests setting up reference checks for new temporary workers who have never worked with the agency.,The client suggests setting up reference checks for new temporary workers who have never worked with the agency.,8.318791,4.293978
3,"Temporary workers present deficiencies in terms of discipline at work, impacting the quality of their performance and their compliance with processes.","Temporary workers present deficiencies in terms of discipline at work, impacting the quality of their performance and their compliance with processes.",7.730595,1.098164
4,It is necessary to improve the accuracy of SI/SIR declarations based on the exposures and risks associated with each workstation for better traceability and effective monitoring of the health of temporary workers throughout and after their career.,It is necessary to improve the accuracy of SI/SIR declarations based on the exposures and risks associated with each workstation for better traceability and effective monitoring of the health of temporary workers throughout and after their career.,7.237301,3.729509
...,...,...,...,...
292,The customer requests an increase in responsiveness from the company.,Customers are dissatisfied due to a lack of proactive communication and insufficient responsiveness from the company.,7.380105,5.209806
293,It is suggested to set up a service available 24 hours a day.,"Customers want more direct and accessible personal interactions, with simplified communication through a dedicated point of contact and continuously available services.",7.860811,5.051519
294,Clients are very satisfied with the quality of the temporary workers offered by Randstad.,Clients express high overall satisfaction with the quality and professionalism of talent provided by Randstad.,4.776226,3.920316
295,"The customer expressed overall satisfaction, stating that their experience was “very good.”","Customers greatly appreciate the professionalism, the quality of listening and the excellence of the interactions with the Randstad advisor, reflecting highly satisfactory customer service.",3.126764,4.765698


In [14]:

# Plot
chart = alt.Chart(df_explore).mark_circle(size=60).encode(
    x=#'x',
    alt.X('x',
        scale=alt.Scale(zero=False)
    ),
    y=
    alt.Y('y',
        scale=alt.Scale(zero=False)
    ),
    color='InsightParent',
    tooltip=['text', 'InsightParent']
).properties(
    width=700,
    height=400
)
chart.interactive()

alt.Chart(...)

In [15]:
df_explore

,text,InsightParent,x,y
0,"Temporary workers lack discipline in terms of punctuality, professionalism, and attention to detail, which results in costly mistakes and inappropriate behavior at work.","Temporary workers present deficiencies in terms of discipline at work, impacting the quality of their performance and their compliance with processes.",7.751192,0.942769
1,"Candidates report not being informed about the company, its business and job details before the interview, creating a lack of suitability and preparation.","Clients are dissatisfied with the selection of temporary workers, linked to candidates feeling poorly informed about the positions before the interview, leading to a profile/position mismatch.",7.273573,2.097942
2,The client suggests setting up reference checks for new temporary workers who have never worked with the agency.,The client suggests setting up reference checks for new temporary workers who have never worked with the agency.,8.318791,4.293978
3,"Temporary workers present deficiencies in terms of discipline at work, impacting the quality of their performance and their compliance with processes.","Temporary workers present deficiencies in terms of discipline at work, impacting the quality of their performance and their compliance with processes.",7.730595,1.098164
4,It is necessary to improve the accuracy of SI/SIR declarations based on the exposures and risks associated with each workstation for better traceability and effective monitoring of the health of temporary workers throughout and after their career.,It is necessary to improve the accuracy of SI/SIR declarations based on the exposures and risks associated with each workstation for better traceability and effective monitoring of the health of temporary workers throughout and after their career.,7.237301,3.729509
...,...,...,...,...
292,The customer requests an increase in responsiveness from the company.,Customers are dissatisfied due to a lack of proactive communication and insufficient responsiveness from the company.,7.380105,5.209806
293,It is suggested to set up a service available 24 hours a day.,"Customers want more direct and accessible personal interactions, with simplified communication through a dedicated point of contact and continuously available services.",7.860811,5.051519
294,Clients are very satisfied with the quality of the temporary workers offered by Randstad.,Clients express high overall satisfaction with the quality and professionalism of talent provided by Randstad.,4.776226,3.920316
295,"The customer expressed overall satisfaction, stating that their experience was “very good.”","Customers greatly appreciate the professionalism, the quality of listening and the excellence of the interactions with the Randstad advisor, reflecting highly satisfactory customer service.",3.126764,4.765698


In [16]:
df_explore["Insight"] = 1

In [17]:
other_sentences = ["Les gens sont mécontents", "Les gens sont très contents", "Le produit est parfait", "Le produit doit être amélioré", "délais", "réactivité", "incohérence"]
other_sentence_embeddings = model.encode(other_sentences)

In [18]:
# Prepare the data to plot and interactive visualization
# using Altair


In [19]:
other_umap_embeds = reducer.transform(other_sentence_embeddings)

other_df = pd.DataFrame({"text":other_sentences, 'x': other_umap_embeds[:,0], 'y': other_umap_embeds[:,1], "Insight": 0})
other_df

,text,x,y,Insight
0,Les gens sont mécontents,5.445783,2.045481,0
1,Les gens sont très contents,5.434158,1.879970,0
2,Le produit est parfait,5.206202,1.879840,0
3,Le produit doit être amélioré,5.326531,2.125319,0
4,délais,5.478425,1.760025,0
5,réactivité,5.397023,1.929666,0
6,incohérence,5.367457,2.063822,0


In [20]:
df_explore_new = pd.concat([df_explore, other_df])

In [21]:
# Plot
chart = alt.Chart(df_explore_new).mark_circle(size=60).encode(
    x=#'x',
    alt.X('x',
        scale=alt.Scale(zero=False)
    ),
    y=
    alt.Y('y',
        scale=alt.Scale(zero=False)
    ),
    tooltip=['text'],
    color='Insight',
).properties(
    width=700,
    height=400
)
chart.interactive()

alt.Chart(...)

In [22]:
df_explore_new.head()

,text,InsightParent,x,y,Insight
0,"Temporary workers lack discipline in terms of punctuality, professionalism, and attention to detail, which results in costly mistakes and inappropriate behavior at work.","Temporary workers present deficiencies in terms of discipline at work, impacting the quality of their performance and their compliance with processes.",7.751192,0.942769,1
1,"Candidates report not being informed about the company, its business and job details before the interview, creating a lack of suitability and preparation.","Clients are dissatisfied with the selection of temporary workers, linked to candidates feeling poorly informed about the positions before the interview, leading to a profile/position mismatch.",7.273573,2.097942,1
2,The client suggests setting up reference checks for new temporary workers who have never worked with the agency.,The client suggests setting up reference checks for new temporary workers who have never worked with the agency.,8.318791,4.293978,1
3,"Temporary workers present deficiencies in terms of discipline at work, impacting the quality of their performance and their compliance with processes.","Temporary workers present deficiencies in terms of discipline at work, impacting the quality of their performance and their compliance with processes.",7.730595,1.098164,1
4,It is necessary to improve the accuracy of SI/SIR declarations based on the exposures and risks associated with each workstation for better traceability and effective monitoring of the health of temporary workers throughout and after their career.,It is necessary to improve the accuracy of SI/SIR declarations based on the exposures and risks associated with each workstation for better traceability and effective monitoring of the health of temporary workers throughout and after their career.,7.237301,3.729509,1


In [23]:
df_explore_new['text'].to_list()

[' Temporary workers lack discipline in terms of punctuality, professionalism, and attention to detail, which results in costly mistakes and inappropriate behavior at work.',
 ' Candidates report not being informed about the company, its business and job details before the interview, creating a lack of suitability and preparation.',
 ' The client suggests setting up reference checks for new temporary workers who have never worked with the agency.',
 ' Temporary workers present deficiencies in terms of discipline at work, impacting the quality of their performance and their compliance with processes.',
 'It is necessary to improve the accuracy of SI/SIR declarations based on the exposures and risks associated with each workstation for better traceability and effective monitoring of the health of temporary workers throughout and after their career.',
 ' The client appreciates the professionalism during his visit to Valencia and suggests the creation of an HR support service offering vari

In [24]:
df_explore_new['embedding'] = reducer.transform(df_explore_new['text'])

ValueError: could not convert string to float: ' Temporary workers lack discipline in terms of punctuality, professionalism, and attention to detail, which results in costly mistakes and inappropriate behavior at work.'

In [ ]:
umap_embeds

array([[ 2.7892613 ,  4.7492185 ],
       [ 5.859206  ,  1.0253093 ],
       [ 5.5827217 ,  1.1252486 ],
       [ 2.3357992 ,  2.0950499 ],
       [-0.29825407,  5.843427  ],
       [ 4.9516044 ,  1.7748865 ],
       [ 1.1163592 ,  5.597881  ]], dtype=float32)